In [1]:
import numpy as np
import pandas as pd
import warnings
from math import isnan
import re

warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('../Output/0505/train_data_tezheng1.csv', encoding='utf-8')
test = pd.read_csv('../Output/0505/test_data_tezheng1.csv', encoding='utf-8')

In [3]:
train.shape

(38199, 399)

In [4]:
# 将训练集中指标为NAN的舍弃掉

train = train[~train.收缩压.isnull()]
train = train[~train.舒张压.isnull()]

In [5]:
train.shape

(38192, 399)

In [6]:
num_cols = list(train.describe().columns[5:])   # 纯数字+NAN

In [7]:
main_cols = ['vid','收缩压', '舒张压', '血清甘油三酯', '血清高密度脂蛋白', '血清低密度脂蛋白']  # vid和指标字段
digit_cols1 = ['1814', '190', '191', '2403', '2404', '2405']                                    # BMI指标和数字（含未查等特殊）
# digit_cols2 = ['10004','10002','1115','1117','1815','183','1850','192','193','2174','314']      # 纯数字型 （float+str）
wenben_cols = ['0409', '0434', 'B701','A801','A705', '2302', '0114','3401','1001','0102','A202',
               '0403','0441','0131','0912','1316','1402','0978','0954']                        # 文本
emu_cols = ['3190', '3191', '3192', '3195', '3196','3197','3430',
            '3189','3194','3485','3486','360']                              # 枚举型
num_and_wenben_cols = ['3193','1840','A701','A703', '0424']                                          # 数字型 （含>= 阴性等特殊）

In [8]:
# 查找混合型字段
# 如 >100次/分,窦性心动过速    ------简单混合型
digit_cols2 = []
exist_digit_cols = main_cols + digit_cols1 + num_and_wenben_cols
p = re.compile('\d+\.?\d*')
r = train.shape[0]
for c in train.columns:
    nulls = train[c].isnull().sum()
    a = train[c].apply(lambda x: 1 if len(p.findall(str(x))) != 0 else 0).sum()
    if (a+nulls) > r*0.9 and a > r*0.08 and c not in exist_digit_cols:
        digit_cols2.append(c)
print (len(digit_cols2)) # 0.95 0.1: 127     0.9 0.05: 207   0.9 0.08 165

165


In [9]:
# 枚举类型在10-30之间的字段
emu_columns = []
exist_emu_cols = digit_cols2 + emu_cols + wenben_cols + ['0125','0440','0973','0981','1304','3207','3399','3402','30007']
for c in train.columns:
    types = len(train[c].unique())
    if types<30 and types >10 and c not in exist_emu_cols:
        emu_columns.append(c)

print (len(emu_columns))

16


In [10]:
select_cols = main_cols + num_cols + digit_cols1 + wenben_cols + emu_cols + num_and_wenben_cols + digit_cols2 + emu_columns

In [11]:
train1 = train[[c for c in train.columns if c in select_cols]]
test1 = test[[c for c in test.columns if c in select_cols]]

In [12]:
# # 寻找某些特定指标相关的疾病涉及字段
# search_cols = []
# for c in train.columns:
#     a = train[c].apply(lambda x: 1 if '甲亢' in str(x) else 0).sum()
#     if a > 100:
#         search_cols.append(c)
# search_cols

In [13]:
# print (train['30007'].unique())
# print (len(train['30007'].unique()))
# print (train['30007'].isnull().sum())

In [14]:
# eye_labels = ['exit','未见明显异常','不肿大','无','nan','未查','未见异','未触及','弃检','弃查','未见明显异常,建议b超检查','眼底弃查']
# test['1402'].apply(lambda x: str(x) if  '未见明显异常' not in str(x) and '不正常' in str(x) and '未见异常' not in str(x) else 0).unique()

# 文本字段处理

In [15]:
def heartModify(x):
    x = str(x)
    if '心界扩大' in x:
        return 1
    return 0

def hasOperate(x):
    x = str(x)
    if '开颅术' in x:
        return 1
    elif '子宫' in x or '卵巢' in x:
        return 2
    elif '甲状腺' in x:
        return 3
    elif '心脏' in x:
        return 4
    return 0
    

#0403 0441 0131
def processGanyouRelateFea(data):
    train = data.copy()
    train['0403'] = train['0403'].apply(heartModify)
    train['0441'] = train['0441'].apply(hasOperate)
    train['0131'] = train['0131'].apply(lambda x: 1 if '未见明显异常' not in str(x) else 0)
    
    normal_label = ['exit','未见明显异常','不肿大','无','nan','未查','未见异','未触及','弃检','弃查','未见明显异常,建议b超检查','眼底弃查']
    train['0912'] = train['0912'].apply(lambda x: 1 if  str(x) not in normal_label and '未见异常' not in str(x) else 0)
    train['0978_jiazhuangxian'] = train['0978'].apply(lambda x: 1 if  '甲状腺' in str(x) else 0)
    train['0954_jiazhuangxian'] = train['0954'].apply(lambda x: 1 if  '甲状腺' in str(x) else 0)
    train['1316'] = train['1316'].apply(lambda x: 1 if  '正常' not in str(x) and '未见异常' not in str(x) and str(x) not in normal_label else 0)
    train['1402'] = train['1402'].apply(lambda x: 1 if  '未见明显异常' not in str(x) and '正常' in str(x) and '未见异常' not in str(x) else 0)
    
    train = train.drop(['0978','0954'], axis=1)
    return train

train2 = processGanyouRelateFea(train1)
test2 = processGanyouRelateFea(test1)

In [16]:
def label_Bchao(x):
    x = str(x)
    if '绝经' in x:
        return 0
    if '高血压' in x:
        return 1
    if '脂肪肝' in x:
        return 2
    else:
        return 3
    
def processWenBen(data):
    train = data.copy()
    cols = ['0102','A202']
    for col in cols:
        train[col+'bchao'] = train[col].apply(label_Bchao)
    return train.drop(cols, axis=1)

test2 = processWenBen(test2)
train2 = processWenBen(train2)

In [17]:
def xindiantu(x):
    x = str(x)
    if x == 'nan' or '正常心电图' in x:
        return 0
    return 1

def process1001(data):
    train = data.copy()
    train['1001_xindiantu'] = train['1001'].apply(xindiantu)
    return train.drop(['1001'], axis=1)

test2 = process1001(test2)
train2 = process1001(train2)

In [18]:
def process114_2302(data):
    train = data.copy()
    train['0114_normal'] = (train['0114'].fillna(-1)).apply(lambda x: int('胆囊大小、形态正常' in  x) if x!= -1 else x)
    train = train.drop(['0114'], axis=1)

    train['2302_jiankang'] = (train['2302'].fillna(-1)).apply(lambda x: int('健康' in  x) if x!= -1 else x)
    train = train.drop(['2302'], axis=1)
    return train

test2 = process114_2302(test2)
train2 = process114_2302(train2)

In [19]:
def process409_434(data):
    train = data.copy()
    cols = []
    for c in ['0409', '0434']:
        train[c] = train[c].astype('str')
        if c == '0434':
            train[c+'_smoke'] = train[c].apply(lambda x: '吸烟' in  x).astype('int')
            train[c+'_gaoxueya'] = train[c].apply(lambda x: '高血压' in  x or '血压偏高' in x).astype('int')
            train[c+'_gaoxuezhi'] = train[c].apply(lambda x: '血脂偏高' in  x ).astype('int')
            train[c+'_tangniaobing'] = train[c].apply(lambda x: '糖尿病' in  x ).astype('int')
            train[c+'_niaosuan'] = train[c].apply(lambda x: '尿酸偏高' in  x ).astype('int')
            train[c+'_gan'] = train[c].apply(lambda x: '肝' in  x).astype('int')
            train[c+'_zhifangan'] = train[c].apply(lambda x: '脂肪肝' in  x).astype('int')
            train[c+'_feipang'] = train[c].apply(lambda x: '肥胖' in  x).astype('int')
            train[c+'_heart'] = train[c].apply(lambda x: '心' in  x or '冠状动脉' in x).astype('int')
            train[c+'_pregnancy'] = train[c].apply(lambda x: '孕' in  x and '宫外孕术后' not in x).astype('int')
            train[c+'_yanzheng'] = train[c].apply(lambda x: '炎' in  x).astype('int')
            train[c+'_jzxkangjin'] = train[c].apply(lambda x: '甲亢' in  x or '甲状腺功能亢进'in x).astype('int')
            train[c+'_jzxjt'] = train[c].apply(lambda x: '甲状腺功能减退' in  x).astype('int')
        else:
            train[c+'_free'] = train[c].apply(lambda x: '未发现明显异常' in  x or '未见异常' in x).astype('int')
            train[c+'_yanzheng'] = train[c].apply(lambda x: '炎' in  x).astype('int')
            train[c+'_niaosuan'] = train[c].apply(lambda x: '高尿酸' in  x ).astype('int')
            train[c+'_gaoxuezhi'] = train[c].apply(lambda x: '血脂偏高' in  x ).astype('int')
            train[c+'_gaoxuetang'] = train[c].apply(lambda x: '血糖偏高' in  x ).astype('int')
            train[c+'_gaoxueya'] = train[c].apply(lambda x: '高血压' in  x or '血压偏高' in x).astype('int')
            train[c+'_gaoxuezhi'] = train[c].apply(lambda x: '血脂偏高' in  x or '高血脂' in x).astype('int')
            train[c+'_zhifangan'] = train[c].apply(lambda x: '脂肪肝' in  x).astype('int')
            train[c+'_beatslow'] = train[c].apply(lambda x: '心动过缓' in  x).astype('int')
            train[c+'_beatfast'] = train[c].apply(lambda x: '心动过速' in  x).astype('int')
            train[c+'_beatunregular'] = train[c].apply(lambda x: '心律不齐' in  x).astype('int')
            train[c+'_beatfre'] = train[c].apply(lambda x: '偶发早搏' in  x).astype('int')
            train[c+'_heartok'] = train[c].apply(lambda x: '心率正常' in  x).astype('int')
            train[c+'_heartnoise'] = train[c].apply(lambda x: '心脏杂音' in  x or '心音' in x).astype('int')
            train[c+'_tangniaobing'] = train[c].apply(lambda x: '糖尿病' in  x ).astype('int')
            train[c+'_gan'] = train[c].apply(lambda x: '肝' in  x).astype('int')
            train[c+'_heart'] = train[c].apply(lambda x: '心肌' in  x or '心脏' in x).astype('int')
            train[c+'_jzxkangjin'] = train[c].apply(lambda x: '甲亢' in  x or '甲状腺功能亢进'in x).astype('int')
            train[c+'_jzxjt'] = train[c].apply(lambda x: '甲状腺功能减退' in  x).astype('int')
            train[c+'_tangniaobing'] = train[c].apply(lambda x: '糖尿病' in  x ).astype('int')

    return train.drop(['0409','0434'], axis=1)

test2 = process409_434(test2)
train2 = process409_434(train2)

In [20]:
def getFeature3401(x):
    x = str(x)
    if x == 'nan' or ('未见明显异常' in x and '检查' not in x and '复查'not in x):
        return 0
    elif '退变' in x :
        return 2
    elif '未见明显异常' in x and ('检查'  in x or '复查' in x):
        return 1
    else:
        return 3

def process3401(data):
    train = data.copy()
    train['3401_ct'] = train['3401'].apply(getFeature3401)
    return train.drop(['3401'], axis=1)

test2 = process3401(test2)
train2 = process3401(train2)

In [21]:
def processAB(data):
    train = data.copy()
    train['A705_label'] = (train['A705'].fillna(-1)).apply(lambda x: int('数字化肝超未发现明显异常' in  x) if x != -1 else x)
    train['A801_label'] = (train['A801'].fillna(-1)).apply(lambda x: int('脑血流图未发现明显异常' in  x) if x != -1 else x)
    train['B701_label'] = (train['B701'].fillna(-1)).apply(lambda x: int('糖尿病或糖尿病前期风险较小' in  x) if x != -1 else x)
    train = train.drop(['B701','A801','A705'], axis=1)
    return train

test3 = processAB(test2)
train3 = processAB(train2)

3402基本无异常

A601大部分详见纸质报告

数字为主： A701、A703、3193、1840  

性别标签: 0120 0121 0125 0130

# 枚举类型字段处理

枚举型变量：

'3190', '3191', '3192', '3195', '3196','3197'   ：阴阳性，+-

混合型变量：

 '0424','1840','3193'

In [22]:
# 枚举型'3190', '3191', '3192', '3195', '3196','3197',3430
def emuFeature(x):
    good_labels = ['nan','阴性','未做','normal','正常']
    nor_labels = ['+-','未见']
    bad_labels = ['阳性','阳性(+)','阳性(1+)','阳性3+']
    x = str(x)
    if x in nor_labels or '+-' in x:
        return 1
    elif x in good_labels or '-' in x:
        return 0
    elif x in bad_labels or '+' in x:
        return 2
    else:
        return 0
    
def processEmu(data):
    train = data.copy()
    for col in emu_cols:
        train[col] =  train[col].apply(emuFeature)
        
    return train

test3 = processEmu(test3)
train3 = processEmu(train3)

In [23]:
def numAndWenben(x):
    x = str(x)
    if x == 'nan' or x =='阴性' or x=='-' or x=='未做':
        return np.nan
    else:
        p = re.compile('\d+\.?\d*')
        return p.findall(x)[0]

def processNumAndWenben(train):
    data = train.copy()
    cols = ['3193','1840','A701','A703']
    for c in cols:
        data[c] = data[c].apply(numAndWenben).astype('float').fillna(0)
    return data

test3 = processNumAndWenben(test3)
train3 = processNumAndWenben(train3)

In [24]:
def processEmuWenFeature(data):
    train = data.copy()
    emu_good_labels = ['nan', '未见异常','无异常','正常','未闻及异常','有力',
                   '弃查','未触及','未 触及','未及','不大','exit','未见明显异常',
                  '视不见','未查','无压痛','无','未见异','纹理清晰']
    for c in emu_columns:
        train[c] = train[c].apply(lambda x: 0 if '无' in str(x) or '未' in str(x) or str(x) in emu_good_labels else 1)
        
    return train

test4 = processEmuWenFeature(test3)
train4 = processEmuWenFeature(train3)

# 数值型字段处理

In [25]:
def get_heart_rate(x):
    import re
    result = re.findall(r"\d+",x)
    if result != []:
        return result[0]
    else :
        return 0

def is_heart_slow(x):
    return ('过缓' in x) or ('<' in x )or (int(get_heart_rate(x))<60 and int(get_heart_rate(x))>1)

def is_heart_fast(x):
    return ('过速' in x) or ('>' in x )or (int(get_heart_rate(x))>100 ) 

def is_heart_normal(x):
    return (int(get_heart_rate(x))>60 and int(get_heart_rate(x))<100) or ('正常' in x) or ('未见' in x)

def process424(data):
    train = data.copy()
    train['0424'] = train['0424'].astype('str')
    train['424_heart_rate'] = (train['0424'].apply(lambda x: get_heart_rate(x))).astype('int')
    train['424_heart_slow'] = train['0424'].apply(lambda x: is_heart_slow(x)).astype('int')
    train['424_heart_fast'] = train['0424'].apply(lambda x: is_heart_fast(x)).astype('int')
    train['424_heart_normal'] = train['0424'].apply(lambda x: is_heart_normal(x)).astype('int')

    return train.drop(['0424'], axis=1)

test4 = process424(test4)
train4 = process424(train4)

In [26]:
def modifyYiChang(x):
    if ';' in str(x):
        spices = x.split(';')
        if spices.count('nan') == 2 or spices.count('未查') == 2 or spices.count('弃查') == 2:
            return np.nan
        elif spices.count('nan') == 1:
            return spices[1-spices.index('nan')]
        elif spices.count('未查') == 1:
            return spices[1-spices.index('未查')]
        elif spices.count('弃查') == 1:
            return spices[1-spices.index('弃查')]
        else :
            return spices[0]
    elif '查' in str(x):
        return np.nan
    return x

def process4(data):
    train = data.copy()
    for col in digit_cols1:
        train[col] = train[col].apply(modifyYiChang).astype('float').fillna(0)
    return train

test4 = process4(test4)
train4 = process4(train4)

In [27]:
def searchDigitRe(x):
    x = str(x)
    if x == 'nan':
        return 0
    else:
        try:
            p = re.compile('\d+\.?\d*')
            return p.findall(x)[0]
        except:
            return 0

def processPureDigit(data):
    train = data.copy()
    for col in num_cols:
            train[col] = train[col].apply(searchDigitRe).astype('float').fillna(0)
    return train

test4 = processPureDigit(test4)
train4 = processPureDigit(train4)

In [28]:
def processDigit2(data):
    train = data.copy()
    for col in digit_cols2:
        train[col] = train[col].apply(searchDigitRe).astype('float').fillna(0)
    return train

test_end = processDigit2(test4)
train_end = processDigit2(train4)

In [42]:
train_end.to_csv('../Output/0505/train_data_tezheng2.csv', index=False, encoding='utf-8')
test_end.to_csv('../Output/0505/test_data_tezheng2.csv', index=False, encoding='utf-8')

In [29]:
train_end.shape

(38192, 282)

In [58]:
train1 = train_end
test1 = test_end

————————————————————————————————————————————————————————————

In [1]:
import numpy as np
import pandas as pd
import warnings
from math import isnan
import re

warnings.filterwarnings("ignore")


train1 = pd.read_csv('../Output/0505/train_data_tezheng2.csv', encoding='utf-8')
test1 = pd.read_csv('../Output/0505/test_data_tezheng2.csv', encoding='utf-8')

In [2]:
train = pd.read_csv('../Output/0505/train_data_tezheng1.csv', encoding='utf-8')
num_cols = list(train.describe().columns[5:])   # 纯数字+NAN

main_cols = ['vid','收缩压', '舒张压', '血清甘油三酯', '血清高密度脂蛋白', '血清低密度脂蛋白']  # vid和指标字段
digit_cols1 = ['1814', '190', '191', '2403', '2404', '2405']                                    # BMI指标和数字（含未查等特殊）
# digit_cols2 = ['10004','10002','1115','1117','1815','183','1850','192','193','2174','314']      # 纯数字型 （float+str）
wenben_cols = ['0409', '0434', 'B701','A801','A705', '2302', '0114','3401','1001','0102','A202',
               '0403','0441','0131','0912','1316','1402','0978','0954']                        # 文本
emu_cols = ['3190', '3191', '3192', '3195', '3196','3197','3430',
            '3189','3194','3485','3486','360']                              # 枚举型
num_and_wenben_cols = ['3193','1840','A701','A703', '0424']   

# 查找混合型字段
# 如 >100次/分,窦性心动过速    ------简单混合型
digit_cols2 = []
exist_digit_cols = main_cols + digit_cols1 + num_and_wenben_cols
p = re.compile('\d+\.?\d*')
r = train.shape[0]
for c in train.columns:
    nulls = train[c].isnull().sum()
    a = train[c].apply(lambda x: 1 if len(p.findall(str(x))) != 0 else 0).sum()
    if (a+nulls) > r*0.9 and a > r*0.05 and c not in exist_digit_cols:
        digit_cols2.append(c)
print (len(digit_cols2)) # 0.95 0.1: 127     0.9 0.05: 207

# 枚举类型在10-30之间的字段
emu_columns = []
exist_emu_cols = digit_cols2 + emu_cols + wenben_cols + ['0125','0440','0973','0981','1304','3207','3399','3402','30007']
for c in train.columns:
    types = len(train[c].unique())
    if types<30 and types >10 and c not in exist_emu_cols:
        emu_columns.append(c)

print (len(emu_columns))

207
14


In [29]:
train1.shape

(38192, 282)

In [59]:
# 比赛要求自定义log_loss
def cal_logloss(label, pre_label):
    rows = len(label)
    log_sum = sum((np.log(pre_label+1) - np.log(label+1)) ** 2)
    return log_sum / rows


In [60]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [61]:
digit_cols = num_cols + ['424_heart_rate','1814', '190', '191', '2403', '2404', '2405'] + digit_cols2 + ['3193','1840','A701','A703']

In [62]:
# 归一化
rows = train1.shape[0]
data = train1.append(test1)
data[['424_heart_rate','1814', '190', '191', '2403', '2404', '2405']] = data[['424_heart_rate','1814', '190', '191', '2403', '2404', '2405']].fillna(0)
for c in digit_cols:
    data[c] = MinMaxScaler().fit_transform(data[c].values.reshape(-1, 1))

In [63]:
train1 = data[:rows]
test1 = data[rows:]
train1.shape

(38192, 282)

In [64]:
shousuoya_label = train1.pop('收缩压')
shuzhangya_label = train1.pop('舒张压')
highzhidanbai_label = train1.pop('血清高密度脂蛋白')
ganyou_label = train1.pop('血清甘油三酯')
lowzhidanbai_label = train1.pop('血清低密度脂蛋白')

train1 = train1.drop('vid', axis =1)

In [65]:
test1.pop('收缩压')
test1.pop('舒张压')
test1.pop('血清高密度脂蛋白')
test1.pop('血清甘油三酯')
test1.pop('血清低密度脂蛋白')

test1_vid = test1.pop('vid')

In [66]:
train_table = {'收缩压':shousuoya_label,
              '舒张压':shuzhangya_label,
              '血清甘油三酯':ganyou_label,
              '血清高密度脂蛋白':highzhidanbai_label,
              '血清低密度脂蛋白':lowzhidanbai_label}

In [80]:
log_score = {}
test_label = {}
test_size_dict = {'收缩压':0.05,
                  '舒张压':0.01,
                  '血清甘油三酯':0.2,
                  '血清高密度脂蛋白':0.2,
                  '血清低密度脂蛋白':0.01}

emu_columns   
emu_cols 

digit_cols1 = ['1814', '190', '191', '2403', '2404', '2405']

digit_cols2 = ['10004','10002','1115','1117','1815','183','1850','192','193','2174','314']

wenben_cols = ['0409', '0434', 'B701','A801','A705', '2302', '0114','3401','1001','0102','A202',
               '0403','0441','0131','0912','1316','1402','0978','0954']
               
emu_cols = ['3190', '3191', '3192', '3195', '3196','3197','3430',
            '3189','3194','3485','3486','360']
            
num_and_wenben_cols = ['3193','1840','A701','A703', '0424'] 

In [67]:
LGBM = lgb.LGBMRegressor(objective='regression_l2', 
                         max_depth=7, 
                         n_estimators=60, 
                         learning_rate=0.1, 
                         num_leaves=63, 
                         min_samples_split=2)

In [68]:
def xunlian(data, train_table=train_table):
    train = data.copy()
    log_score = 0
    for key in train_table.keys():
        X_train,X_test,y_train,y_test = train_test_split(train2, train_table[key], test_size=0.2, random_state=0)
        LGBM.fit(X_train, y_train)
        preLabel = LGBM.predict(X_test)
        log_cur = cal_logloss(y_test, preLabel)
        log_score += log_cur
        log_avg = log_score / 5
    return log_avg

# train2 = train1[[c for c in train1.columns if c not in select_cols_fanwei]]
xunlian(train1)

0.032630458206609834

In [69]:
select_cols_fanwei = wenben_cols + emu_cols + num_and_wenben_cols + digit_cols1 + ['0441','0978_jiazhuangxian','0131','1316'] + emu_columns
current_cols = [c for c in train1.columns if c not in select_cols_fanwei]
select_cols = []
best_score = xunlian(train1[[c for c in current_cols]])
print (best_score)
for row in select_cols_fanwei:
    current_cols.append(row)
    train2 = train1[[c for c in train1.columns if c in current_cols]]
    log_avg = xunlian(train2)
    if best_score <= log_avg:
        current_cols.pop()
    else:
        print (str(row) + ':'+ str(log_avg))
        best_score = log_avg
        select_cols.append(row)
            
print (select_cols)

0.0326304582066
0409:0.0326304582066
0434:0.0326304582066
B701:0.0326304582066
A801:0.0326304582066
A705:0.0326304582066
2302:0.0326304582066
0114:0.0326304582066
3401:0.0326304582066
1001:0.0326304582066
0102:0.0326304582066


KeyboardInterrupt: 

In [52]:
drop_cols = ['0441','0978_jiazhuangxian','0131','1316'] + num_and_wenben_cols + emu_columns +['0403','0441','0131','0912','1316','1402','0978','0954']
log_score = 0
test_label = {}
train2 = train1[[c for c in train1.columns if c not in drop_cols]]
test2 = test1[[c for c in test1.columns if c not in drop_cols]]
for key in train_table.keys():
    X_train,X_test,y_train,y_test = train_test_split(train2, train_table[key], test_size=0.2, random_state=0)
    LGBM.fit(X_train, y_train)
    preLabel = LGBM.predict(X_test)
    log_cur = cal_logloss(y_test, preLabel)
    log_score += log_cur
#     preLabel = LGBM.predict(test2)
#     test_label[key] = preLabel
    print (log_cur)
print (log_score / 5)

0.0146691616043
0.0186938486299
0.0776254080108
0.0126093343976
0.0342919036802
0.0315779312646


In [66]:
def generateResult(vid, shousuoya_label, shuzhangya_label, ganyou_label, highzhidanbai_label, lowzhidanbai_label, filename='../Output/result0505.csv'):
    result = pd.DataFrame({'vid':vid})
    result['收缩压'] = shousuoya_label
    result['舒张压'] = shuzhangya_label
    result['血清甘油三酯'] = ganyou_label
    result['血清高密度脂蛋白'] = highzhidanbai_label
    result['血清低密度脂蛋白'] = lowzhidanbai_label
    result.to_csv(filename, index=False, header=False ,line_terminator='\r')

In [138]:
generateResult(test1_vid, test_label['收缩压'], test_label['舒张压'], 
               test_label['血清甘油三酯'], test_label['血清高密度脂蛋白'], 
               test_label['血清低密度脂蛋白'], filename='../Output/new_result312.csv')

# 收缩压

In [92]:
key = '收缩压'
drop_cols = ['0441','0978_jiazhuangxian','0131','1316'] + num_and_wenben_cols
train2 = train1[[c for c in train1.columns if c not in drop_cols]]
test2 = test1[[c for c in test1.columns if c not in drop_cols]]

LGBM = lgb.LGBMRegressor(objective='regression_l2', 
                         max_depth=7, 
                         n_estimators=600, 
                         learning_rate=0.01, 
                         num_leaves=63, 
                         min_samples_split=2,
                         subsample=0.7)

X_train,X_test,y_train,y_test = train_test_split(train2, train_table[key], test_size=test_size_dict[key], random_state=1)
LGBM.fit(X_train, y_train)
preLabel = LGBM.predict(X_test)
log_cur = cal_logloss(y_test, preLabel)
log_score.setdefault(key, 0.0)
log_score[key] = log_cur
preLabel = LGBM.predict(test2)
test_label[key] = preLabel
print (log_cur)

0.0138741132151


# 舒张压

In [110]:
key = '舒张压'
drop_cols = ['0441','0978_jiazhuangxian','0131','1316']
train2 = train1[[c for c in train1.columns if c not in drop_cols]]
test2 = test1[[c for c in test1.columns if c not in drop_cols]]

LGBM = lgb.LGBMRegressor(objective='regression_l2', 
                         max_depth=7, 
                         n_estimators=600, 
                         learning_rate=0.01, 
                         num_leaves=63, 
                         min_samples_split=2,
                         subsample=0.7)


X_train,X_test,y_train,y_test = train_test_split(train2, train_table[key], test_size=test_size_dict[key], random_state=47)
LGBM.fit(X_train, y_train)
preLabel = LGBM.predict(X_test)
log_cur = cal_logloss(y_test, preLabel)

log_score.setdefault(key, 0.0)
log_score[key] = log_cur
preLabel = LGBM.predict(test2)
test_label[key] = preLabel
print (log_cur)

0.0139016007101


# 血清甘油三酯

In [112]:
key = '血清甘油三酯'
drop_cols = ['0441','0978_jiazhuangxian','0131','1316'] + num_and_wenben_cols + emu_columns
train2 = train1[[c for c in train1.columns if c not in drop_cols]]
test2 = test1[[c for c in test1.columns if c not in drop_cols]]

LGBM = lgb.LGBMRegressor(objective='regression_l2', 
                         max_depth=7, 
                         n_estimators=600, 
                         learning_rate=0.01, 
                         num_leaves=63,
                         min_samples_split=2)

X_train,X_test,y_train,y_test = train_test_split(train2, train_table[key], test_size=test_size_dict[key], random_state=34)
LGBM.fit(X_train, y_train)
preLabel = LGBM.predict(X_test)
log_cur = cal_logloss(y_test, preLabel)
log_score.setdefault(key, 0.0)
log_score[key] = log_cur
preLabel = LGBM.predict(test2)
test_label[key] = preLabel
print (log_cur)

0.0751216670762


# 血清高密度脂蛋白

In [115]:
key = '血清高密度脂蛋白'
drop_cols = ['0441','0978_jiazhuangxian','0131','1316'] + emu_cols
train2 = train1[[c for c in train1.columns if c not in drop_cols]]
test2 = test1[[c for c in test1.columns if c not in drop_cols]]

LGBM = lgb.LGBMRegressor(objective='regression_l2', 
                         max_depth=7, 
                         n_estimators=60, 
                         learning_rate=0.1, 
                         num_leaves=63,
                         min_samples_split=2)

X_train,X_test,y_train,y_test = train_test_split(train2, train_table[key], test_size=test_size_dict[key], random_state=2)
LGBM.fit(X_train, y_train)
preLabel = LGBM.predict(X_test)
log_cur = cal_logloss(y_test, preLabel)

log_score.setdefault(key, 0.0)
log_score[key] = log_cur
preLabel = LGBM.predict(test2)
test_label[key] = preLabel
print (log_cur)

0.0120046143224


# 血清低密度脂蛋白

In [118]:
key = '血清低密度脂蛋白'
drop_cols = ['0441','0978_jiazhuangxian','0131','1316'] + num_and_wenben_cols + emu_columns
train2 = train1[[c for c in train1.columns if c not in drop_cols]]
test2 = test1[[c for c in test1.columns if c not in drop_cols]]

LGBM = lgb.LGBMRegressor(objective='regression_l2', 
                         max_depth=7, 
                         n_estimators=600, 
                         learning_rate=0.01, 
                         num_leaves=63,
                         min_samples_split=2)

X_train,X_test,y_train,y_test = train_test_split(train2, train_table[key], test_size=test_size_dict[key], random_state=13)
LGBM.fit(X_train, y_train)
preLabel = LGBM.predict(X_test)
log_cur = cal_logloss(y_test, preLabel)

log_score.setdefault(key, 0.0)
log_score[key] = log_cur
preLabel = LGBM.predict(test2)
test_label[key] = preLabel
print (log_cur)

0.0267328307068


In [119]:
score = 0
for val in log_score.values():
    score += val
score / 5

0.02832696520611443

In [120]:
log_score.values()

dict_values([0.013874113215116947, 0.013901600710077497, 0.075121667076236825, 0.012004614322355112, 0.026732830706785761])

In [35]:
test_label = {}
for key in train_table.keys():
    LGBM.fit(train1, train_table[key])
    preLabel = LGBM.predict(test1)
    test_label[key] = preLabel

In [121]:
generateResult(test1_vid, test_label['收缩压'], test_label['舒张压'], 
               test_label['血清甘油三酯'], test_label['血清高密度脂蛋白'], 
               test_label['血清低密度脂蛋白'], filename='../Output/new_result0507.csv')